In [1]:
import pandas as pd
import tensorflow as tf  
import os  
import numpy as np  
import cv2  
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
from tensorflow.keras.utils import to_categorical

In [2]:
path=r'C:\Users\saurabh\Documents\Desktop\Neural Network and Deep Learning\Mask Detection'
X=np.load(os.path.join(path,'ImagesArray.npy'))
Y=np.load(os.path.join(path,'ImageLabelsArray.npy'))

In [3]:
Y_encoded= label_encoder.fit_transform(Y)
y = to_categorical(Y_encoded, 3)

In [4]:
from tensorflow import keras
model = keras.models.load_model(os.path.join(path,'maskclassifiermodel.h5'))

In [ ]:
face_cascade = cv2.CascadeClassifier(os.path.join(path,'haarcascade_frontalface_default.xml'))
vid = cv2.VideoCapture(0)
while(True):
    ret, image = vid.read()
    image = cv2.flip(image, 1)
    face_rect = face_cascade.detectMultiScale(image,
                                              scaleFactor = 1.2,
                                              minNeighbors = 7)
    for (x1, y1, w, h) in face_rect:
        faceImage=image[y1:y1+h, x1:x1+w,:]
        faceImage=cv2.resize(faceImage,(40,40))
        faceImage=faceImage/255.0
        faceImage=faceImage.reshape(1,40,40,3)
        PredictfaceImage=model.predict([faceImage])[0]
        Predictedlabel=label_encoder.inverse_transform([np.where(PredictfaceImage==max(PredictfaceImage))[0][0]])[0] 
        PredictedlabelChance=round(max(PredictfaceImage)*100,2)
        text='{}: {}%'.format(Predictedlabel,PredictedlabelChance)
        
        
        
        
        image=cv2.rectangle(image, (x1, y1),
                  (x1 + w, y1 + h), (255, 255, 255), 5)
        
        image = cv2.putText(image, text, (x1,y1),  cv2.FONT_HERSHEY_SIMPLEX, 
               0.6, (0, 0, 255), 1, cv2.LINE_AA)
    
    cv2.imshow('MaskDetector', image)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

vid.release()
cv2.destroyAllWindows()
